# Hướng dẫn sử dụng 
- Nhập file XLSX: từ folder xlsxLink/[name].xlsx
- Chọn folder output: outputFolder/[name].csv

In [9]:
# Config:
xlsx_file = 'xlsxLink/MoisturizersBarrierTherapeutics.xls.xlsx'
# Slpit
version = 1
start = 0
end = start + 600

csv_output = f'outputFolder/MoisturizersBarrierTherapeutics{version}.csv'

In [10]:
import time

import pandas as pd

# transform xls to dataframe
df = pd.read_excel(xlsx_file)

In [11]:
# Install selenium if you haven't already
# !pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [12]:
import pandas as pd
from collections import defaultdict

# Initialize an empty dictionary to store the grouped data
grouped_data = defaultdict(list)

#TODO: CHeck some link URLS not correct position

# Go to the specific URL
from tqdm import tqdm

df_split = df[start:end]

for url in tqdm(df_split['URL'], desc="Processing URLs"):  # only the first 10 links
    time.sleep(6)  # sleep 5s
    driver.get(url)

    # Cloudflare protection
    # If the page is protected by Cloudflare, timesleep 10s to user tick on browser

    # Find the ul tag with class name 'fa-ul'
    try:
        ul_element = driver.find_element(By.CLASS_NAME, 'fa-ul')
    except:
        continue
    # For each li tag in the ul tag
    for li in ul_element.find_elements(By.TAG_NAME, 'li'):
        # Find all a tags in the li tag
        a_tags = li.find_elements(By.TAG_NAME, 'a')

        for a in a_tags:
            # Get name
            name = a.text.strip()  # Optional: strip to remove leading/trailing whitespaces
            # Get href attribute
            href = a.get_attribute('href')

            # Group URLs by name
            grouped_data[name].append(href)

# Convert the grouped data into a DataFrame
result_data = pd.DataFrame({
    "Name": grouped_data.keys(),
    "URLs": grouped_data.values()
})

Processing URLs: 100%|██████████| 600/600 [1:12:24<00:00,  7.24s/it]


In [13]:
# Close the browser
driver.quit()

In [14]:
# export result to csv
result_data.to_csv(csv_output)